In [1]:
import pickle
import pandas as pd
import sqlite3

features_df = pickle.load(open('features_df_norm.pckl','rb'))

song_df = pickle.load(open('full_songs_df.pckl','rb'))

In [2]:
len(features_df)

4675

In [3]:
import sqlite3

conn = sqlite3.connect('songs_db.db')
print('opened database successfully')

opened database successfully


In [4]:
def songQuery(title, num_songs):
    import sqlite3
    conn = sqlite3.connect('songs_db.db')
    
    title_insert = '%' + title + '%'
    
    matches =  conn.execute("""SELECT TITLE FROM SONGS WHERE TITLE LIKE (?)""", (title_insert,)).fetchall()
    if len(matches) == 0:
        print( 'Try another query!' )
        return
    
    elif len(matches) > 1:
        for i in range(len(matches)):
            print(i+1,'-', matches[i][0])
            
        print('')    
        selection = int(input('Which entry matches your desired song? '))
        
        if selection in range(len(matches) + 1):
            pass
        else:
            print('Try another query!')
            return 
            
        desired_song = conn.execute("""SELECT ID FROM SONGS WHERE TITLE LIKE (?)""", (matches[selection-1][0],)).fetchall()
    
    else:
        desired_song = conn.execute("""SELECT ID FROM SONGS WHERE TITLE LIKE (?)""", (title_insert,)).fetchall()
        
    makePlaylist(matches[selection-1][0], desired_song[0][0], num_songs)

In [9]:
def makePlaylist(title,songID, num_songs):

    from sklearn.metrics import pairwise_distances
    import shutil, os
    import string

    min_dists = pairwise_distances(features_df.loc[songID].values.reshape(1,-1), features_df.values)
    neighbor_indices = min_dists.argsort()[0][:num_songs]
    
    id_list = []
    
    for neighbors in neighbor_indices:
        id_list += [features_df.iloc[neighbors].name]
        
    new_playlist = song_df[song_df.index.isin(id_list)]
    new_playlist_entries = new_playlist['title']
    #print(new_playlist_entries, len(new_playlist_entries))
    
    wav_list = []
    
    for i in id_list:
        wav_list += [i + '.wav']
        
    print('\nThe {0} songs most similar to {1} are: \n'.format(num_songs, title))
        
    music_dir = 'D:\Documents\Music\wav_files'
    folder = 'D:\Documents\Music\Playlists' + '\\' + title + '\\'
    
    os.mkdir(folder)
        
    for f in range(len(wav_list)):
        song_copy = music_dir + '\\' + wav_list[f]
        song_rename = folder + new_playlist_entries[f] + '.wav'
        folder_new = folder + wav_list[f]
        print(new_playlist_entries[f])
        shutil.copyfile(song_copy,folder_new)


    for f in range(len(wav_list)):
        new_song_copy = folder + wav_list[f]
        song_copy = music_dir + '\\' + wav_list[f]
        
        full_subj = new_playlist_entries[f]
        subj = full_subj.replace('.','').replace('?','').replace(':','').replace('!','').replace('+','').replace('\"','').replace('*','').replace('/','').replace('|','')
        
        song_rename = folder + subj + '.wav'
        os.rename(new_song_copy,song_rename)
        #print('rename success', f)
    
    return new_playlist_entries # if a dataframe of the related titles is desired.

In [10]:
a = songQuery('RIT', 20)

1 - VÉRITÉ - Somewhere In Between (Lyric Video)
2 - VÉRITÉ - Solutions (Audio)
3 - VÉRITÉ - Need Nothing (Lyric Video)
4 - Lemaitre - Iron Pyrite
5 - VÉRITÉ - Saint (EDES Remix)
6 - 【Drum&Bass】Tritonal - Anchor (Noisestorm Remix)
7 - Tritonal - Call Me (Asketa Remix)
8 - VÉRITÉ - Saint (tofû Remix)
9 - Liam Payne, Rita Ora - For You (Fifty Shades Freed)
10 - Tritonal + Sj - Calabasas [Lyric Video]
11 - Tritonal & Sj – Hung Up Ft. Emma Gatsby [Lyric Video]
12 - Allie X – Casanova ft. VÉRITÉ
13 - Tritonal - Good Thing (Young In Mind Remix)
14 - Rita Ora - Anywhere (Official Video)
15 - Frances - When It Comes To Us (feat. RITUAL)
16 - Tritonal - Call Me [Lyric Video]
17 - AWAY - Honest To Gød (feat. Charity)
18 - Tritonal ft. Laurell - Good Thing (Justin Caruso Remix)
19 - Ritual - The Fall
20 - Jynjo - How do you Like it (feat. Spritely)
21 - Tritonal feat. Laurell - Good Thing
22 - Avicii - Lonely Together ft. Rita Ora
23 - The Chainsmokers - Honest (Tritonal Remix)
24 - VÉRITÉ - Bett